In [ ]:
import json
from typing import List, Dict

import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration


class ImageCaptioningAgent:
    """AI agent that extracts image crops from viewport screenshots and
    produces natural‑language captions for each using BLIP.
    """

    def __init__(self, device: str | None = None):
        """Load BLIP model + processor and move to the chosen device.

        Args:
            device: Optional forced device string ("cuda", "cpu", "mps", …).
                    Defaults to the first available CUDA device, else CPU.
        """
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        # Salesforce/blip-image-base is a base‑size vision‑language model that
        # supports zero‑shot image captioning.
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-base")
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-base")
        self.model.to(self.device)
        self.model.eval()

    def preprocess(self, raw_json: str) -> List[Dict]:
        """Parse the incoming JSON and crop all bounding boxes.

        Each **viewport** entry supplies a full‑page ``screenshot`` path and an
        ``image_captioning`` array with ``nodeId``, ``alt`` text, and a ``bbox``.
        For every item we load the screenshot exactly once per viewport, crop
        the box, and stash the crop as a **PIL.Image** so that the next stage
        can batch/loop over them.
        """
        doc = json.loads(raw_json)

        crops: List[Dict] = []
        for vp in doc.get("viewports", []):
            screenshot_path: str | None = vp.get("screenshot")
            if not screenshot_path:
                # Skip viewports that do not provide a screenshot path.
                continue

            try:
                page_img = Image.open(screenshot_path).convert("RGB")
            except Exception as exc:  # pragma: no cover — I/O safeguard
                raise RuntimeError(f"Failed to open screenshot {screenshot_path}: {exc}") from exc

            for entry in vp.get("image_captioning", []):
                node_id = entry.get("nodeId")
                alt_text = entry.get("alt", "")
                bbox = entry.get("bbox", {})

                # Basic sanity ‑ fallback to 0 for any missing coordinate.
                x, y = bbox.get("x", 0), bbox.get("y", 0)
                w, h = bbox.get("width", 0), bbox.get("height", 0)

                # Guard against degenerate boxes — BLIP will error on size 0.
                if w <= 0 or h <= 0:
                    continue

                crop = page_img.crop((x, y, x + w, y + h))
                crops.append({
                    "nodeId": node_id,
                    "alt": alt_text,
                    "image": crop,
                })

        if not crops:
            raise ValueError("No image crops found in the provided JSON")

        return crops

    @torch.inference_mode()
    def generate_summary(
        self,
        crops: List[Dict],
        max_length: int = 20,
        num_beams: int = 4,
    ) -> List[tuple[str, str, str]]:
        """Run BLIP captioning over every crop.

        Returns a list of ``(nodeId, alt, caption)`` tuples suitable for the
        formatter in ``handle``.
        """
        results: List[tuple[str, str, str]] = []
        for item in crops:
            encoded = self.processor(images=item["image"], return_tensors="pt").to(self.device)
            out_ids = self.model.generate(
                **encoded,
                max_length=max_length,
                num_beams=num_beams,
                early_stopping=True,
            )
            caption = self.processor.decode(out_ids[0], skip_special_tokens=True).strip()
            results.append((item["nodeId"], item["alt"], caption))
        return results

    def handle(self, raw_json: str) -> str:
        """Top‑level entry: JSON → captions in the requested plain‑text format."""
        crops = self.preprocess(raw_json)
        triples = self.generate_summary(crops)

        # Assemble the exact output phrasing required by the spec.
        sentences: list[str] = []
        for node_id, alt, caption in triples:
            alt_text = alt if alt else "no alternate text provided"
            sentences.append(
                f"For nodeId {node_id}, the alt image text is '{alt_text}', and the generated caption is '{caption}'."
            )

        return " ".join(sentences)